# 13장 데이터 분석 예제
* https://github.com/wesm/pydata-book

## [13.2 무비렌즈의 영화 평점 데이터]
* 1990년대 말부터 2000년대 movieLens 사용자 6000여 명에게서 수집한 4000여 편의 영화 평점 1M 데이터 제공
  * 사용자 정보: 나이, 우편번호, 성별, 직업
  * 영화 평점 정보 : 장르, 개봉 연도
  * 영화 정보 : 장르, 제목
- Gender :
  * "M" : 남성
  * "F" : 여성
- Age :
	*  1:  "Under 18"
	* 18:  "18-24"
	* 25:  "25-34"
	* 35:  "35-44"
	* 45:  "45-49"
	* 50:  "50-55"
	* 56:  "56+"
- Occupation :
	*  0:  "other" or not specified
	*  1:  "academic/educator"
	*  2:  "artist"
	*  3:  "clerical/admin"
	*  4:  "college/grad student"
	*  5:  "customer service"
	*  6:  "doctor/health care"
	*  7:  "executive/managerial"
	*  8:  "farmer"
	*  9:  "homemaker"
	* 10:  "K-12 student"
	* 11:  "lawyer"
	* 12:  "programmer"
	* 13:  "retired"
	* 14:  "sales/marketing"
	* 15:  "scientist"
	* 16:  "self-employed"
	* 17:  "technician/engineer"
	* 18:  "tradesman/craftsman"
	* 19:  "unemployed"
	* 20:  "writer"
- Genres :
	* Action
	* Adventure
	* Animation
	* Children's
	* Comedy
	* Crime
	* Documentary
	* Drama
	* Fantasy
	* Film-Noir
	* Horror
	* Musical
	* Mystery
	* Romance
	* Sci-Fi
	* Thriller
	* War
	* Western


In [ ]:
# 1. 필요한 라이브러리 임포트
import pandas as pd

In [ ]:
# 데이터셋 로드
unames = ["user_id", "gender", "age", "occupation", "zip"]
users = pd.read_table("users.dat", sep="::",
                      header=None, names=unames, engine="python")

rnames = ["user_id", "movie_id", "rating", "timestamp"]
ratings = pd.read_table("ratings.dat", sep="::",
                        header=None, names=rnames, engine="python")

mnames = ["movie_id", "title", "genres"]
movies = pd.read_table("movies.dat", sep="::",
                       header=None, names=mnames, engine="python")

In [ ]:
# 데이터셋 탐색
print(users.head())
print(ratings.head())
print(movies.head())

## [데이터 분석 문제]

### # 성별에 따른 250건 이상의 평균 평점 정보가 있는 영화 추출하기

In [ ]:
# (1) ratings 테이블과 users 테이블을 병합하기 -> 그 결과를 movies 테이블과 병합하기 : 1000209 rows × 10 columns
data = pd.merge(pd.merge(ratings, users), movies)
data

## pandas.pivot_table()
* 주어진 데이터프레임을 특정 열을 기준으로 그룹화하기
* 다양한 집계 함수를 사용하여 요약 통계를 계산
* 주요 파라미터
  * data : 피벗 테이블을 생성할 데이터프레임
  * values : 집계할 데이터 열
  * index: 피벗 테이블의 행으로 사용할 열
  * columns: 피벗 테이블의 열로 사용할 열
  * aggfunc: 집계함수 목록
  * fill_value: 결측값을 대체할 값

In [ ]:
# (2) 성별에 따른 각 영화의 평균 평점을 집계하기
mean_ratings = pd.pivot_table(data = data, values = "rating", index="title",
                                columns="gender", aggfunc="mean")
mean_ratings.head()

In [ ]:
# (3) 영화 제목별 건수 집계하기
ratings_by_title = data.groupby("title").size()
ratings_by_title.head()

In [ ]:
# (4) 250건 이상의 평점 정보가 있는 영화 제목 추출
active_titles = ratings_by_title.index[ratings_by_title >= 250]
active_titles

In [ ]:
# (5) 성별에 따른 250건 이상의 평점 정보가 있는 영화에 대한 색인 추출하기
mean_ratings = mean_ratings.loc[active_titles]
mean_ratings

### # 여성에게서 높은 평점을 받은 영화 목록 확인하기
* 성별에 따른 250건 이상의 평점 정보가 있는 영화

In [ ]:
top_female_ratings = mean_ratings.sort_values("F", ascending=False)
top_female_ratings.head()

### # 남성에게서 높은 평점을 받은 영화 목록 확인하기
* 성별에 따른 250건 이상의 평점 정보가 있는 영화

In [ ]:
top_male_ratings = mean_ratings.sort_values("M", ascending=False)
top_male_ratings.head()

### # 남녀 간의 호불호가 갈리는 영화를 찾기
* 성별에 따른 250건 이상의 평점 정보가 있는 영화


In [ ]:
# (1) mean_ratings에 평균 평점의 차이를 계산하기
mean_ratings["diff"] = mean_ratings["M"] - mean_ratings["F"]

In [ ]:
# (2) 여성이 선호하는 영화 순서대로 정렬
sorted_by_diff = mean_ratings.sort_values("diff")
sorted_by_diff.head()

In [ ]:
# (3) 역순으로 정렬한 다음 남성이 선호하는 영화 확인하기
sorted_by_diff = mean_ratings.sort_values("diff", ascending=False)
sorted_by_diff.head()

### # 성별에 관계없이 영화에 대한 호불호가 극명하게 나뉘는 영화 찾기
* 호불호는 평점의 표준편차을 이용하여 측정
* 250건 이상의 평점 정보가 있는 영화

In [ ]:
#(1) 영화별로 평점의 표준편차를 계산하고 평점이 250건 이상인 영화만 선택하기
rating_std_by_title = data.groupby("title")["rating"].std()
rating_std_by_title = rating_std_by_title.loc[active_titles]
rating_std_by_title.head()

In [ ]:
# (2) 내림차순으로 정렬하여 호불호가 극명하게 갈리는 영화 10개 선정
rating_std_by_title.sort_values(ascending=False)[:10]

### # 장르와 연령별로 그룹하여 평균 평점을 구하기

In [ ]:
# 여러 장르에 속하는 영화일 경우 장르를 개별적으로 분리하기
movies["genre"] = movies.pop("genres").str.split("|")
movies.head()

In [ ]:
# (2) 복수 장르에 속하는 영화을 각 장르별로 나누기
movies_exploded = movies.explode("genre")
movies_exploded[:10]

In [ ]:
# (3) 세 테이블을 모두 병합하여 그룹화하기
ratings_with_genre = pd.merge(pd.merge(movies_exploded, ratings), users)
ratings_with_genre.head(20)

In [ ]:
# (4) 장르와 연령별로 그룹하여 평균 평점을 구하기
genre_ratings = (ratings_with_genre.groupby(["genre", "age"])
                 ["rating"].mean().unstack("age")
                 )
genre_ratings[:10]

## [데이터사이언스]
* 선형회귀 분석
  * 각 장르별로 연령대와 평점 간의 관계를 분석하는 선형 회귀 모델을 구축하고, 그 성능을 비교
  * age를 독립 변수로, 각 장르별 평점을 종속 변수로 사용
  * from sklearn.linear_model import LinearRegression

In [ ]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

# 데이터프레임 생성
df = genre_ratings
df.describe()

# age와 각 장르의 평점을 각각 독립 변수와 종속 변수로 설정
X = np.array([1, 18, 25, 35, 45, 50, 56]).reshape(-1, 1)
y = df.loc[genre].values 

# 각 장르에 대해 선형 회귀 모델을 훈련하고 평가
model = LinearRegression()
model.fit(X, y)
y_pred = model.predict(X)

# 모델의 성능을 MSE와 R^2 스코어로 측정
mse = mean_squared_error(y, y_pred)
r2 = r2_score(y, y_pred)

results.append({
    'genre': genre,
    'mse': mse,
    'r2': r2
})


# subplots을 이용하여 각 장르에 대해 선형 회귀 모델을 훈련하고 실제값과 예측값 시각화






# 모든 장르의 결과를 데이터프레임으로 정리



# 결과 데이터프레임을 이용하여 MSE와 R^2 스코어를 막대그래프로 시각화




## [데이터사이언스]
* 로지스틱회귀 분석
  * 각 장르별로 연령대와 평점 간의 관계를 "good("0") & bad("1")로 분류분석하는 로지스틱 회귀 모델을 구축하고, 그 성능을 비교
  * age를 독립 변수
  * 각 장르별 평점을 종속 변수 - 이진 변수로 설정
  * from sklearn.linear_model import LogisticRegression

In [ ]:
# 분류문제로 변경
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt

# 데이터 셋 : 데이터프레임 생성
df = genre_ratings

# 데이터 전처리
X = np.array([1, 18, 25, 35, 45, 50, 56]).reshape(-1, 1)
threshold = 3.5

# 각 장르에 대해 로지스틱 회귀 모델을 훈련하고 평가결과 시각화
results = []
num_genres = len(df.index)
num_rows = (num_genres + 1) // 2

fig, axs = plt.subplots(nrows=num_rows, ncols=2, figsize=(15, 5 * num_rows))
axs = axs.flatten()

for idx, genre in enumerate(df.index):
    # 연속형 종속변수의 범주형 변수로 변환(이진변수)


    
    # 로지스틱 회귀 모형의 지도학습 및 예측

    

    # 모델의 성능을 정확도, 혼동 행렬 및 분류 보고서로 측정



    # 모델의 성능 평가 결과을 데이토프레임에 저장

    
    # 각 장르에 대해 실제 값과 예측 값을 시각화
    axs[idx].scatter(X, y, color='blue', label="actual")
    axs[idx].plot(X, model.predict(X), color='red', label="predicted")
    axs[idx].set_title(f'{genre} - Actual vs Predicted')
    axs[idx].set_xlabel('Age')
    axs[idx].set_ylabel('Rating')
    axs[idx].legend()

plt.tight_layout()
plt.show()

# 모든 장르의 결과를 데이터프레임으로 정리



# 모든 장르의 정확도를 막대그래프로 시각화
